In [2]:
import mujoco
import gym
import tianshou as ts
import numpy as np

/Users/qkatlehdrnf/opt/anaconda3/envs/opai39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
env = gym.make('custom_env:XiaoTian-v0')
env.reset()

/Users/qkatlehdrnf/opt/anaconda3/envs/opai39/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:64: UserWarning: WARN: A Box action space maximum and minimum values are equal.
  logger.warn("A Box action space maximum and minimum values are equal.")


(array([-0.08844834,  0.99423349,  0.02522243,  0.04961169, -0.09166388,
         0.05169569,  0.09740672, -0.07451653, -0.0453078 ,  0.06623219,
         0.02131772,  0.04698502,  0.08980745, -0.07644738,  0.08400598,
         0.02799501, -0.04792939,  0.04569674, -0.07739866, -0.03838727,
        -0.03277548,  0.09526785,  0.05365187,  0.0504775 , -0.0279894 ,
        -0.24485   ,  0.0537131 , -0.07676325, -0.03612536,  0.01226874,
         0.02797546,  0.08457968, -0.18778297,  0.02922114, -0.04459452,
         0.06539261,  0.04044155,  0.07328328, -0.07929681, -0.13205696,
         0.05311957, -0.00215206, -0.12678795, -0.01520035, -0.01222321,
        -0.01858538, -0.15636012,  0.10649006,  0.01368705,  0.08390033,
        -0.00172834]),
 {})

custom_env:XiaoTian-v0
custom_env:XiaoTian-v0

In [9]:
env.render()

/Users/qkatlehdrnf/opt/anaconda3/envs/opai39/lib/python3.9/site-packages/gym/envs/mujoco/mujoco_env.py:363: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("XiaoTian-v0", render_mode="rgb_array")
  gym.logger.warn(


In [27]:
env.action_space

Box(0.0, 0.0, (20,), float32)

In [3]:
train_envs = gym.make('custom_env:XiaoTian-v0')
test_envs = gym.make('custom_env:XiaoTian-v0')


In [7]:
train_envs = ts.env.DummyVectorEnv(
    [lambda: gym.make('custom_env:XiaoTian-v0') for _ in range(10)])
test_envs = ts.env.DummyVectorEnv(
    [lambda: gym.make('custom_env:XiaoTian-v0') for _ in range(100)])


/Users/qkatlehdrnf/opt/anaconda3/envs/opai39/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:64: UserWarning: WARN: A Box action space maximum and minimum values are equal.
  logger.warn("A Box action space maximum and minimum values are equal.")


DQN이 왜 안되냐면 일단 기본적으로 Argmax를 이용하게 되는데 그러면 net의 return이 2**20이 되야 한다.

In [14]:
import torch as tc
import numpy as np
from torch import nn


class ActorNet(nn.Module):
    def __init__(self, state_shape, action_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(state_shape), 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape)),
        )

    def forward(self, obs, state=None, info={}):
        if not isinstance(obs, tc.Tensor):
            obs = tc.tensor(obs, dtype=tc.float)
        batch = obs.shape[0]
        logits = self.model(obs.view(batch, -1))
        return logits, state

class CriticNet(nn.Module):
    def __init__(self, state_shape, action_shape):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(np.prod(state_shape) + np.prod(action_shape), 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 128), nn.ReLU(inplace=True),
            nn.Linear(128, 1),
        )

    def forward(self, obs, act, state=None, info={}):
        if not isinstance(obs, tc.Tensor):
            obs = tc.tensor(obs, dtype=tc.float)
        if not isinstance(act, tc.Tensor):
            act = tc.tensor(act, dtype=tc.float)
        batch = obs.shape[0]
        x = tc.cat([obs.view(batch, -1), act.view(batch, -1)], dim=1)
        logits = self.model(x)
        return logits


state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
ActorNet = ActorNet(state_shape, action_shape)
CriticNet = CriticNet(state_shape, action_shape)
Aoptim = tc.optim.Adam(ActorNet.parameters(), lr=1e-3)
Coptim = tc.optim.Adam(CriticNet.parameters(), lr=1e-3)

In [32]:
CriticNet

CriticNet(
  (model): Sequential(
    (0): Linear(in_features=71, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [15]:
policy = ts.policy.DDPGPolicy(actor=ActorNet, actor_optim=Aoptim, critic=CriticNet, critic_optim=Coptim, estimation_step=3)


In [33]:
train_collector = ts.data.Collector(
    policy, train_envs, ts.data.VectorReplayBuffer(200000, 10), exploration_noise=True)
test_collector = ts.data.Collector(policy, test_envs, exploration_noise=True)


In [1]:
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector,
    max_epoch=100, step_per_epoch=10000, step_per_collect=10,
    update_per_step=0.1, episode_per_test=15, batch_size=64,
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold)
print(f'Finished training! Use {result["duration"]}')


NameError: name 'ts' is not defined

In [18]:
from torch.utils.tensorboard import SummaryWriter
from tianshou.utils import TensorboardLogger
writer = SummaryWriter('log/dqn')
logger = TensorboardLogger(writer)


In [21]:
tc.save(policy.state_dict(), 'dqn.pth')
policy.load_state_dict(tc.load('dqn.pth'))

<All keys matched successfully>

In [24]:
policy.eval()
# policy.set_eps(0.05)
collector = ts.data.Collector(policy, env, exploration_noise=True)
collector.collect(n_episode=1, render=1 / 35)

/Users/qkatlehdrnf/opt/anaconda3/envs/opai39/lib/python3.9/site-packages/tianshou/data/collector.py:68: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")
/Users/qkatlehdrnf/opt/anaconda3/envs/opai39/lib/python3.9/site-packages/gym/envs/mujoco/mujoco_env.py:363: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("XiaoTian-v0", render_mode="rgb_array")
  gym.logger.warn(


{'n/ep': 1,
 'n/st': 1,
 'rews': array([0.88650187]),
 'lens': array([1]),
 'idxs': array([0]),
 'rew': 0.8865018660339451,
 'len': 1.0,
 'rew_std': 0.0,
 'len_std': 0.0}

In [25]:

action = policy(Batch(obs=np.array([obs]))).act[0]

NameError: name 'Batch' is not defined